# Initial Setups


## (Google Colab use only)

In [1]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')
    
    # If there's a package I need to install separately, do it here
    #!pip install pyro-ppl

    # cd to the appropriate working directory under my Google Drive
    %cd 'drive/My Drive/cs696ds_lexalytics/Language Model Finetuning'
    
    # List the directory contents
    !ls

## Experiment ID

**NOTE**: The following `experiment_id` MUST BE CHANGED in order to avoid overwriting the files from other experiments!!!!!!

In [1]:
# We will use the following string ID to identify this particular (training) experiments
# in directory paths and other settings
experiment_id = 'lm_further_pretraining_bert_yelp_restaurants'

## Package Install

In [3]:
# Install packages specified in requirements
!pip install -r requirements.txt

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [4]:
# NVidia APEX install
!unzip -o apex-master_downloaded_01Mar2021.zip
%cd apex-master
!pip install -v --no-cache-dir ./
%cd ..

Archive:  apex-master_downloaded_01Mar2021.zip
e2083df5eb96643c61613b9df48dd4eea6b07690
  inflating: apex-master/.gitignore  
  inflating: apex-master/.gitmodules  
 extracting: apex-master/.nojekyll   
  inflating: apex-master/LICENSE     
  inflating: apex-master/README.md   
 extracting: apex-master/apex/RNN/README.md  
  inflating: apex-master/apex/RNN/RNNBackend.py  
  inflating: apex-master/apex/RNN/__init__.py  
  inflating: apex-master/apex/RNN/cells.py  
  inflating: apex-master/apex/RNN/models.py  
  inflating: apex-master/apex/__init__.py  
  inflating: apex-master/apex/amp/README.md  
  inflating: apex-master/apex/amp/__init__.py  
  inflating: apex-master/apex/amp/__version__.py  
  inflating: apex-master/apex/amp/_amp_state.py  
  inflating: apex-master/apex/amp/_initialize.py  
  inflating: apex-master/apex/amp/_process_optimizer.py  
  inflating: apex-master/apex/amp/amp.py  
  inflating: apex-master/apex/amp/compat.py  
  inflating: apex-master/apex/amp/frontend.py  
 

  inflating: apex-master/apex/contrib/test/multihead_attn/test_fast_self_multihead_attn_bias.py  
  inflating: apex-master/apex/contrib/test/multihead_attn/test_mha_fused_softmax.py  
  inflating: apex-master/apex/contrib/test/multihead_attn/test_self_multihead_attn.py  
  inflating: apex-master/apex/contrib/test/multihead_attn/test_self_multihead_attn_norm_add.py  
  inflating: apex-master/apex/contrib/test/test_label_smoothing.py  
  inflating: apex-master/apex/contrib/xentropy/__init__.py  
  inflating: apex-master/apex/contrib/xentropy/softmax_xentropy.py  
  inflating: apex-master/apex/fp16_utils/README.md  
  inflating: apex-master/apex/fp16_utils/__init__.py  
  inflating: apex-master/apex/fp16_utils/fp16_optimizer.py  
  inflating: apex-master/apex/fp16_utils/fp16util.py  
  inflating: apex-master/apex/fp16_utils/loss_scaler.py  
 extracting: apex-master/apex/mlp/__init__.py  
  inflating: apex-master/apex/mlp/mlp.py  
  inflating: apex-master/apex/multi_tensor_apply/__init__.p

  inflating: apex-master/examples/simple/distributed/run.sh  
  inflating: apex-master/requirements.txt  
  inflating: apex-master/requirements_dev.txt  
  inflating: apex-master/setup.py    
 extracting: apex-master/tests/L0/run_amp/__init__.py  
  inflating: apex-master/tests/L0/run_amp/test_add_param_group.py  
  inflating: apex-master/tests/L0/run_amp/test_basic_casts.py  
  inflating: apex-master/tests/L0/run_amp/test_cache.py  
  inflating: apex-master/tests/L0/run_amp/test_checkpointing.py  
  inflating: apex-master/tests/L0/run_amp/test_fused_sgd.py  
  inflating: apex-master/tests/L0/run_amp/test_larc.py  
  inflating: apex-master/tests/L0/run_amp/test_multi_tensor_axpby.py  
  inflating: apex-master/tests/L0/run_amp/test_multi_tensor_l2norm.py  
  inflating: apex-master/tests/L0/run_amp/test_multi_tensor_scale.py  
  inflating: apex-master/tests/L0/run_amp/test_multiple_models_optimizers_losses.py  
  inflating: apex-master/tests/L0/run_amp/test_promotion.py  
  inflating: ap

  copying build/lib/apex/contrib/multihead_attn/__init__.py -> build/bdist.linux-x86_64/wheel/apex/contrib/multihead_attn
  copying build/lib/apex/contrib/multihead_attn/mask_softmax_dropout_func.py -> build/bdist.linux-x86_64/wheel/apex/contrib/multihead_attn
  copying build/lib/apex/contrib/multihead_attn/encdec_multihead_attn_func.py -> build/bdist.linux-x86_64/wheel/apex/contrib/multihead_attn
  creating build/bdist.linux-x86_64/wheel/apex/RNN
  copying build/lib/apex/RNN/__init__.py -> build/bdist.linux-x86_64/wheel/apex/RNN
  copying build/lib/apex/RNN/cells.py -> build/bdist.linux-x86_64/wheel/apex/RNN
  copying build/lib/apex/RNN/RNNBackend.py -> build/bdist.linux-x86_64/wheel/apex/RNN
  copying build/lib/apex/RNN/models.py -> build/bdist.linux-x86_64/wheel/apex/RNN
  creating build/bdist.linux-x86_64/wheel/apex/fp16_utils
  copying build/lib/apex/fp16_utils/fp16_optimizer.py -> build/bdist.linux-x86_64/wheel/apex/fp16_utils
  copying build/lib/apex/fp16_utils/fp16util.py -> bu

done
  Created wheel for apex: filename=apex-0.1-py3-none-any.whl size=196374 sha256=d0d17254945ddaa7dad1e438eb05366fdc264ccb36cb2df84035c89399f67351
  Stored in directory: /tmp/pip-ephem-wheel-cache-o5nw2o70/wheels/36/a3/d3/fe7e08c0ab558202ffbf166fbdd3ccc3c64d3994d0d58c399c
Successfully built apex
  Attempting uninstall: apex
    Found existing installation: apex 0.1
    Uninstalling apex-0.1:
      Created temporary directory: /home/bseoh/miniconda3/envs/idp01/lib/python3.7/site-packages/~pex-0.1.dist-info
      Removing file or directory /home/bseoh/miniconda3/envs/idp01/lib/python3.7/site-packages/apex-0.1.dist-info/
      Created temporary directory: /home/bseoh/miniconda3/envs/idp01/lib/python3.7/site-packages/~pex
      Removing file or directory /home/bseoh/miniconda3/envs/idp01/lib/python3.7/site-packages/apex/
      Successfully uninstalled apex-0.1

1 location(s) to search for versions of pip:
* https://pypi.org/simple/pip/
Fetching project page and analyzing links: https://

  Found link https://files.pythonhosted.org/packages/00/9e/4c83a0950d8bdec0b4ca72afd2f9cea92d08eb7c1a768363f2ea458d08b4/pip-19.2.3.tar.gz#sha256=e7a31f147974362e6c82d84b91c7f2bdf57e4d3163d3d454e6c3e71944d67135 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*,!=3.4.*), version: 19.2.3
  Found link https://files.pythonhosted.org/packages/4a/08/6ca123073af4ebc4c5488a5bc8a010ac57aa39ce4d3c8a931ad504de4185/pip-19.3-py2.py3-none-any.whl#sha256=e100a7eccf085f0720b4478d3bb838e1c179b1e128ec01c0403f84e86e0e2dfb (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*,!=3.4.*), version: 19.3
  Found link https://files.pythonhosted.org/packages/af/7a/5dd1e6efc894613c432ce86f1011fcc3bbd8ac07dfeae6393b7b97f1de8b/pip-19.3.tar.gz#sha256=324d234b8f6124846b4e390df255cacbe09ce22791c3b714aa1ea6e44a4f2861 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*,!=3.4.*), version: 19.3
  Found link 

Removed build tracker: '/tmp/pip-req-tracker-oib22gla'
/mnt/nfs/work1/696ds-s21/bseoh/lexalytics


In [2]:
# IPython reloading magic
%load_ext autoreload
%autoreload 2

## Package Imports

In [3]:
import sys
import os
import random
import numpy as np
import torch
import transformers
import datasets

import utils

# Random seed settings
random_seed = 696
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("PyTorch version: " + torch.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.7.9 (default, Nov  4 2020, 21:30:07) 
[GCC 7.3.0]
NumPy version: 1.19.2
PyTorch version: 1.7.1
Transformers version: 4.3.3


## PyTorch GPU settings

In [4]:
if torch.cuda.is_available():
    # https://github.com/UKPLab/sentence-transformers/issues/413
    torch.multiprocessing.set_start_method('spawn')
    
    torch_device = torch.device('cuda')

    # Set this to True to make your output immediately reproducible
    # Note: https://pytorch.org/docs/stable/notes/randomness.html
    torch.backends.cudnn.deterministic = False
    
    # Disable 'benchmark' mode: Set this False if you want to measure running times more fairly
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = True
    
    # Faster Host to GPU copies with page-locked memory
    use_pin_memory = True 

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False

print()
print("PyTorch device selected:", torch_device)

CUDA Version: 11.0
cuDNN Version: 8005
CUDA Device Name: GeForce GTX TITAN X
CUDA Capabilities: (5, 2)

PyTorch device selected: cuda


# Further pre-training

## Load the BERT-base model

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir='./bert_base_cache')
model = transformers.AutoModelForMaskedLM.from_pretrained("bert-base-uncased", cache_dir='./bert_base_cache')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load the Yelp restaurants dataset

In [6]:
yelp = datasets.load_dataset(
    './dataset_scripts/yelp_restaurants',
    data_files={
        'train': 'dataset_files/yelp_restaurants/yelp_academic_dataset_review.json',
        'restaurant_ids': 'dataset_files/yelp_restaurants/restaurantIDs.txt',
    },
    cache_dir='./dataset_cache')

Using custom data configuration default
Reusing dataset yelp_restaurants (./dataset_cache/yelp_restaurants/default-ceef9e04fb6ab232/0.0.0/6affa0993e335aaa035f8613a3e49bd465b0dc4ab6d8ea1e7838b5bbc7477077)


In [7]:
data_yelp_train = yelp['train']

In [8]:
print("Number of training data:", len(data_yelp_train))

Number of training data: 2152007


In [9]:
# Check out how individual data points look like
print(data_yelp_train[696])

{'id': 'EUgMR__LDQYjdD6qlWgrlw', 'stars': 1.0, 'text': "Burgers is bland and tasteless, relies on toppings. So expensive. Hot dog is tiny and terribly salty. Fries are similar to Harveys, don't compare to New York Fries. Would not go back."}


### Preprocessing: Encode the text with Tokenizer

In [10]:
train_dataset_pretraining = data_yelp_train.map(
    lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=256),
    remove_columns=data_yelp_train.column_names,
    batched=True, num_proc=4)

Loading cached processed dataset at ./dataset_cache/yelp_restaurants/default-ceef9e04fb6ab232/0.0.0/6affa0993e335aaa035f8613a3e49bd465b0dc4ab6d8ea1e7838b5bbc7477077/cache-3a2a466ed105fd96.arrow
Loading cached processed dataset at ./dataset_cache/yelp_restaurants/default-ceef9e04fb6ab232/0.0.0/6affa0993e335aaa035f8613a3e49bd465b0dc4ab6d8ea1e7838b5bbc7477077/cache-db1767be8460a916.arrow
Loading cached processed dataset at ./dataset_cache/yelp_restaurants/default-ceef9e04fb6ab232/0.0.0/6affa0993e335aaa035f8613a3e49bd465b0dc4ab6d8ea1e7838b5bbc7477077/cache-19859829e41fc129.arrow
Loading cached processed dataset at ./dataset_cache/yelp_restaurants/default-ceef9e04fb6ab232/0.0.0/6affa0993e335aaa035f8613a3e49bd465b0dc4ab6d8ea1e7838b5bbc7477077/cache-90ca4428e74eec72.arrow


## Pre-train further

### Training settings

In [11]:
# Our new 'smart' masking
collator = utils.DataCollatorForSmartMLM(
    tokenizer=tokenizer,
    mlm_probability=0.15,
    prob_replace_with_mask=0.8,
    prob_replace_with_random=0.1,
)

In [12]:
training_args = transformers.TrainingArguments(
    output_dir=os.path.join('.', 'progress', experiment_id, 'results'), # output directory
    overwrite_output_dir=True,
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=32,  # 32 * 2 GPUs = 64 Total
    evaluation_strategy='epoch',
    warmup_steps=5000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=os.path.join('.', 'progress', experiment_id, 'logs'), # directory for storing logs
    seed=random_seed,
    fp16=True,
    fp16_opt_level='O2',
    fp16_backend='apex',
    prediction_loss_only=True,
    load_best_model_at_end=True,
    dataloader_num_workers=22,
)

In [13]:
print(training_args.n_gpu)

2


In [14]:
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=collator, # do the masking on the go
    train_dataset=train_dataset_pretraining,
)

Intel(R) oneAPI Data Analytics Library solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


### Training loop

In [ ]:
%%time
trainer.train()

Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic


### Save the model to the local directory

In [ ]:
trainer.save_model(os.path.join('.', 'trained_models', experiment_id))

In [ ]:
tokenizer.save_pretrained(os.path.join('.', 'trained_models', experiment_id))

## LM Evaluation

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
print(eval_results)

perplexity = np.exp(eval_results["eval_loss"])

print(perplexity)

## Playing with my own input sentences

In [ ]:
example = f"""The {tokenizer.mask_token} of {tokenizer.mask_token} is awful, but its {tokenizer.mask_token} is fantastic."""

example_encoded = tokenizer.encode(example, add_special_tokens=True, return_tensors="pt").to(torch_device)

# Let's decode this back just to see how they were actually encoded
example_tokens = []

for id in example_encoded[0]:
    example_tokens.append(tokenizer.convert_ids_to_tokens(id.item()))

print(example_tokens)

In [ ]:
example_prediction = model(example_encoded)

example_prediction_argmax = torch.argmax(example_prediction[0], dim=-1)[0]

print(example_prediction_argmax)

print(tokenizer.decode(example_prediction_argmax))